In [19]:
import pandas as pd
import random 
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm

import researchpy as rp
import scipy.stats as stats
from scipy.stats import levene
from scipy.stats import kstest
from scipy.stats import skew

import math 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from statsmodels.formula.api import ols
from statsmodels.multivariate.manova import MANOVA


import warnings
warnings.filterwarnings('ignore')

plt.rcParams["figure.figsize"] = (20,10) 
pd.options.display.max_columns = None

In [20]:
df= pd.read_pickle("datos/cars_price_norm_est.pkl")
df.head(2)

,Levy,Manufacturer,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Color,Airbags,Price_log
0,1.475781,LEXUS,-0.411248,Jeep,Yes,Hybrid,3.5,-0.021053,1.375313,Automatic,4x4,04,Silver,1.329406,9.497622
1,0.770541,CHEVROLET,-0.173337,Jeep,No,Petrol,3,-0.020897,1.375313,Tiptronic,4x4,04,Black,0.319668,9.718422


In [6]:
df.dtypes

Levy                 float64
Manufacturer          object
Prod. year           float64
Category              object
Leather interior      object
Fuel type             object
Engine volume         object
Mileage              float64
Cylinders            float64
Gear box type         object
Drive wheels          object
Doors               category
Color                 object
Airbags              float64
Price_log            float64
dtype: object

En el pair programming de hoy usaremos el set de datos que guardastéis en el pair programming de normalización y estandarización.


Hasta ahora habéis estado evaluando las características de vuestro set de datos y habéis hecho una gran exploración, es el momento de hacer vuestro primer ANOVA! En el ejercicio de hoy tendréis que hacer un ANOVA con vuestro datos y hacer una interpretación de los resultados.

📌 NOTA Puede que vuestros datos no se ajusten o no cumplan todas las asunciones, no pasa nada, haced el ANOVA e interpretad los resultados. En próximas lecciones aprenderemos que podemos hacer cuando nos encontramos en esta situación.

In [25]:
df.columns

Index(['Levy', 'Manufacturer', 'Prod. year', 'Category', 'Leather interior',
       'Fuel type', 'Engine volume', 'Mileage', 'Cylinders', 'Gear box type',
       'Drive wheels', 'Doors', 'Color', 'Airbags', 'Price_log'],
      dtype='object')

- hemos detectado que para hacer el anova necesitamos que no haya espacios entre las palabras de los nombres de las columnas, al llegar a este punto, nos damos cuenta que es algo que ya tendriamos que haber modficado al hacer la primera limpieza... :( perooo, más vale tarde que nunca :) 

In [26]:
df.rename(columns={'Prod. year':'Prod_year', 'Leather interior':'Leather_interior', 'Fuel type':'Fuel_type', 'Engine volume':'Engine_volume', 'Gear box type':'Gear_box_type', 'Drive wheels':'Drive_wheels'}, inplace=True)

In [28]:
df.describe()

,Levy,Prod_year,Mileage,Cylinders,Airbags,Price_log
count,9877.000000,9877.000000,9877.000000,9877.000000,9877.000000,13584.000000
mean,0.009686,0.007608,-0.000021,0.002635,0.000912,9.567687
std,1.007079,1.000678,0.995238,1.000446,1.000478,0.982023
min,-1.113801,-2.552451,-0.025895,-3.090707,-1.699806,6.214608
25%,-1.113801,-0.411248,-0.024125,-0.411095,-0.690069,9.198850
50%,0.074556,0.064574,-0.022797,-0.411095,-0.185200,9.699411
75%,0.535461,0.778309,-0.021366,-0.411095,1.329406,10.130185
max,20.569091,1.967865,55.872388,10.307355,2.339143,12.640792


In [27]:
# iniciamos nuestro anova. Recordemos que la variable respuesta es el precio de los coches
lm = ols('Price_log ~ Levy + Manufacturer + Prod_year + Category + Leather_interior + Fuel_type + Engine_volume + Mileage + Cylinders + Gear_box_type + Drive_wheels + Doors + Color + Airbags', data = df).fit()
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
Manufacturer,57.0,857.974596,15.052186,20.674331,4.272742e-196
Category,10.0,367.775275,36.777528,50.514309,1.677094e-99
Leather_interior,1.0,53.544882,53.544882,73.544441,1.134933e-17
Fuel_type,6.0,151.846567,25.307761,34.760468,8.272187e-42
Engine_volume,100.0,780.344324,7.803443,10.718109,1.328323e-152
Gear_box_type,3.0,444.305348,148.101783,203.419308,5.967518e-128
Drive_wheels,2.0,11.319588,5.659794,7.773785,4.232495e-04
Doors,2.0,6.289358,3.144679,4.319249,1.333554e-02
Color,15.0,56.548615,3.769908,5.178007,2.069772e-10
Levy,1.0,0.368008,0.368008,0.505463,4.771267e-01


In [29]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Price_log   R-squared:                       0.279
Model:                            OLS   Adj. R-squared:                  0.264
Method:                 Least Squares   F-statistic:                     19.09
Date:                Thu, 29 Sep 2022   Prob (F-statistic):               0.00
Time:                        22:19:14   Log-Likelihood:                -12348.
No. Observations:                9877   AIC:                         2.509e+04
Df Residuals:                    9680   BIC:                         2.651e+04
Df Model:                         196                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                         9.3327      0.566     16.503      0.000       8.224      10.441
Manufacturer[T.ALFA ROMEO]       -0.2484      0.528     -0.470      0.638      -1.284       0.787
Manufacturer[T.ASTON MARTIN]      0.6597      0.494      1.336      0.181      -0.308       1.627
Manufacturer[T.AUDI]             -0.0372      0.318     -0.117      0.907      -0.661       0.587
Manufacturer[T.BENTLEY]           0.6143      1.092      0.563      0.574      -1.526       2.754
Manufacturer[T.BMW]               0.0271      0.313      0.087      0.931      -0.586       0.640
Manufacturer[T.BUICK]            -0.1854      0.432     -0.429      0.668      -1.032       0.661
Manufacturer[T.CADILLAC]          0.0451      0.426      0.106      0.916      -0.789       0.879
Manufacturer[T.CHEVROLET]         0.2839      0.310      0.916      0.360      -0.324       0.891
Manufacturer[T.CHRYSLER]         -0.0089      0.367     -0.024      0.981      -0.729       0.711
Manufacturer[T.CITROEN]          -0.1076      0.434     -0.248      0.804      -0.958       0.743
Manufacturer[T.DAEWOO]           -0.0337      0.335     -0.101      0.920      -0.691       0.624
Manufacturer[T.DAIHATSU]         -0.7144      0.473     -1.511      0.131      -1.641       0.212
Manufacturer[T.DODGE]            -0.5261      0.330     -1.593      0.111      -1.173       0.121
Manufacturer[T.FERRARI]           2.3580      0.797      2.957      0.003       0.795       3.921
Manufacturer[T.FIAT]              0.2815      0.338      0.832      0.405      -0.382       0.945
Manufacturer[T.FORD]              0.2285      0.310      0.738      0.461      -0.378       0.835
Manufacturer[T.GAZ]              -0.6476      0.910     -0.711      0.477      -2.432       1.137
Manufacturer[T.GMC]              -0.1175      0.423     -0.278      0.781      -0.946       0.711
Manufacturer[T.GREATWALL]     -7.502e-14   1.16e-13     -0.646      0.518   -3.03e-13    1.52e-13
Manufacturer[T.HAVAL]            -0.0286      0.920     -0.031      0.975      -1.832       1.774
Manufacturer[T.HONDA]             0.6275      0.311      2.019      0.043       0.018       1.237
Manufacturer[T.HUMMER]            0.6997      0.619      1.130      0.259      -0.514       1.914
Manufacturer[T.HYUNDAI]           0.7218      0.308      2.341      0.019       0.117       1.326
Manufacturer[T.INFINITI]          0.5089      0.379      1.343      0.179      -0.234       1.252
Manufacturer[T.JAGUAR]            0.5468      0.357      1.530      0.126      -0.154       1.247
Manufacturer[T.JEEP]              0.8210      0.319      2.572      0.010       0.195       1.447
Manufacturer[T.KIA]               0.4784      0.312      1.534      0.125      -0.133       1.090
Manufacturer[T.LANCIA]            0.1538     